DEEP REINFORCEMENT LEARNING EXPLAINED - 10
# **Dynamic Programming**
## V-function in Practice for Frozen-Lake Environment

In [ ]:
import gym
import collections
from torch.utils.tensorboard import SummaryWriter
import numpy as np

ENV_NAME = "FrozenLake-v0"
#ENV_NAME = "FrozenLake8x8-v0"  
GAMMA = 0.95


### The Agent

In [ ]:

class Agent:
    def __init__(self):
        self.env = gym.make(ENV_NAME)
        self.V = np.zeros(self.env.observation_space.n)

    def calc_action_value(self, state, action):
        action_value = sum([prob*(r + GAMMA * self.V[s_])
                          for prob, s_, r, _ in self.env.P[state][action]]) 
        return action_value

    def select_action(self, state):
        best_action, best_value = None, None
        for action in range(self.env.action_space.n):
            action_value = self.calc_action_value(state, action)
            if best_value is None or best_value < action_value:
                best_value = action_value
                best_action = action
        return best_action


    def value_iteration(self):
        for state in range(self.env.observation_space.n):
            state_values = [self.calc_action_value(state, action)
                           for action in range(self.env.action_space.n)
                           ]
            self.V[state] = max(state_values)
        return self.V

### Training loop

In [ ]:
TEST_EPISODES = 40
REWARD_GOAL = 0.90

def train(agent): 
  test_env = gym.make(ENV_NAME)
  writer = SummaryWriter()

  iter_no = 0
  best_reward = 0.0
 
  while best_reward < REWARD_GOAL:

        #step 1
        agent.value_iteration()

        #step 2 check the improvements 
        iter_no += 1
        reward_test = 0.0
        for _ in range(TEST_EPISODES):
            total_reward = 0.0
            state = test_env.reset()
            while True:
                action = agent.select_action(state)
                new_state, new_reward, is_done, _ = test_env.step(action)
                total_reward += new_reward
                if is_done: break
                state = new_state
            reward_test += total_reward
        reward_test /= TEST_EPISODES

        #step track with TensorBoard 
        writer.add_scalar("reward", reward_test, iter_no)
        if reward_test > best_reward:
            print("Best reward updated %.2f at iteration %d " % (reward_test ,iter_no) )
            best_reward = reward_test

  writer.close()

### Training the Agent

In [ ]:
agent = Agent()
train(agent)

Best reward updated 0.28 at iteration 3 
Best reward updated 0.35 at iteration 6 
Best reward updated 0.45 at iteration 7 
Best reward updated 0.55 at iteration 8 
Best reward updated 0.75 at iteration 9 
Best reward updated 0.85 at iteration 18 
Best reward updated 0.88 at iteration 57 
Best reward updated 0.90 at iteration 78 


In [ ]:
def extract_policy(agent):   
    env = gym.make(ENV_NAME)
    policy = np.zeros(env.observation_space.n) 
    for s in range(env.observation_space.n):
        Q_values = [agent.calc_action_value(s,a) for a in range(env.action_space.n)] 
        policy[s] = np.argmax(np.array(Q_values))        
    return policy

def print_policy(policy):
    print(policy.reshape([-1, 4]))
    print("\n")
    a2w = {0:'<', 1:'v', 2:'>', 3:'^'}
    policy_arrows = [a2w[x] for x in policy]
    print(np.array(policy_arrows).reshape([-1, 4]))

In [ ]:
policy=extract_policy(agent)
print_policy(policy)

[[0. 3. 0. 3.]
 [0. 0. 0. 0.]
 [3. 1. 0. 0.]
 [0. 2. 1. 0.]]


[['<' '^' '<' '^']
 ['<' '<' '<' '<']
 ['^' 'v' '<' '<']
 ['<' '>' 'v' '<']]


In [ ]:
env = gym.make(ENV_NAME)
env.env.P[0][0]

[(0.3333333333333333, 0, 0.0, False),
 (0.3333333333333333, 0, 0.0, False),
 (0.3333333333333333, 4, 0.0, False)]

In [ ]:
env.env.P[4][0]

[(0.3333333333333333, 0, 0.0, False),
 (0.3333333333333333, 4, 0.0, False),
 (0.3333333333333333, 8, 0.0, False)]

In [ ]:
env.env.P[8][3]

[(0.3333333333333333, 9, 0.0, False),
 (0.3333333333333333, 4, 0.0, False),
 (0.3333333333333333, 8, 0.0, False)]

### Test the Agent

In [ ]:
def test(agent):
    new_test_env = gym.make(ENV_NAME) 
    state = new_test_env.reset()
    new_test_env.render()
    is_done = False
    iter_no = 0

    while not is_done:
        action = agent.select_action(state)
        new_state, reward, is_done, _ = new_test_env.step(action)
        new_test_env.render()
        state = new_state
        iter_no +=1
    print("reward =    ", reward)
    print("iterations =", iter_no)

test(agent)

In [1]:
%load_ext tensorboard


In [ ]:
tensorboard  --logdir=runs

### The Agent that estimates the Transition Function

In [ ]:
N =100
class Agent_Updated:
    def __init__(self):
        self.env = gym.make(ENV_NAME)
        self.state = self.env.reset()
        self.rewards = collections.defaultdict(float)
        self.transits = collections.defaultdict(
            collections.Counter)
        self.V = np.zeros(self.env.observation_space.n)

    def play_n_random_steps(self, count):
        for _ in range(count):
            action = self.env.action_space.sample()
            new_state, reward, is_done, _ = self.env.step(action)
            self.rewards[(self.state, action, new_state)] = reward
            self.transits[(self.state, action)][new_state] += 1
            if is_done:
                self.state = self.env.reset() 
            else: 
                self.state = new_state

    def calc_action_value(self, state, action):
        target_counts = self.transits[(state, action)]
        total = sum(target_counts.values())
        action_value = 0.0
        for s_, count in target_counts.items():
            r = self.rewards[(state, action, s_)]
            prob = (count / total)
            action_value += prob*(r + GAMMA * self.V[s_])
        return action_value

    def select_action(self, state):
        best_action, best_value = None, None
        for action in range(self.env.action_space.n):
            action_value = self.calc_action_value(state, action)
            if best_value is None or best_value < action_value:
                best_value = action_value
                best_action = action
        return best_action


    def value_iteration(self):
        self.play_n_random_steps(N)
        for state in range(self.env.observation_space.n):
            state_values = [
                self.calc_action_value(state, action)
                for action in range(self.env.action_space.n)
            ]
            self.V[state] = max(state_values)

In [ ]:
agent = Agent_Updated()
train(agent)
policy=extract_policy(agent)
print_policy(policy)

Best reward updated 0.07 at iteration 5 
Best reward updated 0.12 at iteration 7 
Best reward updated 0.30 at iteration 8 
Best reward updated 0.33 at iteration 12 
Best reward updated 0.42 at iteration 13 
Best reward updated 0.60 at iteration 42 
Best reward updated 0.70 at iteration 56 
Best reward updated 0.75 at iteration 66 
Best reward updated 0.80 at iteration 70 
Best reward updated 0.82 at iteration 78 
Best reward updated 0.85 at iteration 80 
Best reward updated 0.88 at iteration 98 
Best reward updated 0.93 at iteration 528 
[[0. 3. 0. 3.]
 [0. 0. 2. 0.]
 [3. 1. 0. 0.]
 [0. 2. 1. 0.]]


[['<' '^' '<' '^']
 ['<' '<' '>' '<']
 ['^' 'v' '<' '<']
 ['<' '>' 'v' '<']]


In [ ]:
tensorboard  --logdir=runs